<a href="https://colab.research.google.com/github/Aashi779/DeepLearningwithPytorch/blob/main/HumanActivityRecognition_ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

os.environ["KAGGLE_USERNAME"] = "ravikanttyagi"
os.environ["KAGGLE_KEY"] = "095a21c789eb4728fde2b29230033273"

!kaggle datasets download meetnagadia/human-action-recognition-har-dataset

100% 296M/297M [00:18<00:00, 19.0MB/s]
100% 297M/297M [00:18<00:00, 17.0MB/s]


In [2]:
from IPython.display import clear_output

!unzip human-action-recognition-har-dataset.zip

clear_output()

In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import torch
from torch import nn
from torchvision import transforms, models
from torch.utils import data
import torchvision.datasets as datasets

In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [5]:
train_path = "Human Action Recognition/train"
test_path = "Human Action Recognition/test"

In [6]:
df = pd.read_csv("Human Action Recognition/Training_set.csv")
class_names = pd.value_counts(df['label']).index
class_names = np.sort(class_names)
print(class_names)

['calling' 'clapping' 'cycling' 'dancing' 'drinking' 'eating' 'fighting'
 'hugging' 'laughing' 'listening_to_music' 'running' 'sitting' 'sleeping'
 'texting' 'using_laptop']


In [7]:
filenames = df['filename'].values

In [8]:
len(filenames)

12600

In [53]:
# transform = transforms.Compose([transforms.ToTensor()])

def load_data(path, df):
  images_list = []
  labels_list = []
  for i in tqdm(range(len(filenames) - 8000)):
    # concat train_path with image name
    img_path = path + "/" + filenames[i]
    # fetch image label from data frame of current image
    img_label = df['label'][i]
    # read image using opencv
    img = cv2.imread(img_path)
    # resize image because images might be of different dimensions
    # in order to maintain array, we have to resize all the images in same dimension
    # img = cv2.resize(img, (150,150))
    # img = transform(img)
    # img = img / 255.0
    # store images one by one in your list
    images_list.append(img)
    labels_list.append(img_label)

  images_arr = np.asarray(images_list)
  labels_arr = np.asarray(labels_list)

  return images_arr, labels_arr

In [54]:
train_images, train_labels = load_data(train_path, df)

100%|██████████| 4600/4600 [00:04<00:00, 979.13it/s] 
<ipython-input-53-ac9a6639ff33>:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  images_arr = np.asarray(images_list)


In [55]:
np.unique(train_labels, return_counts=True)

(array(['calling', 'clapping', 'cycling', 'dancing', 'drinking', 'eating',
        'fighting', 'hugging', 'laughing', 'listening_to_music', 'running',
        'sitting', 'sleeping', 'texting', 'using_laptop'], dtype='<U18'),
 array([322, 312, 315, 296, 307, 313, 307, 316, 332, 272, 290, 317, 315,
        283, 303]))

In [56]:
# Inherit Dataset class coming from data package
class Dataset(data.Dataset):
  def __init__(self, images, labels):
    self.transforms = transforms
    self.images = images
    self.labels = labels

  def __len__(self):
    return len(self.images)

  def __getitem__(self, index):
    # loading data - one image at a time
    X = self.images[index]
    X = cv2.resize(X,(224,224))
    y = self.labels[index]
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        transforms.RandomHorizontalFlip()
        ])
    X = transform(X)
    # X = torch.tensor(X)
    # X = torch.cat((X,X,X),0)
    return X, y

In [57]:
label = LabelEncoder()
train_labels = label.fit_transform(train_labels)

In [58]:
# 75% - training and 25% - testing
x_train, x_test, y_train, y_test = train_test_split(train_images, train_labels, test_size=0.35)

In [59]:
x_train.shape

(2990,)

In [60]:
x_test.shape

(1610,)

In [61]:
params = {"batch_size":16, "shuffle":True}

training_set = Dataset(x_train, y_train)
training_generator = data.DataLoader(training_set, **params)

test_set = Dataset(x_test, y_test)
test_generator = data.DataLoader(test_set, **params)

In [62]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# Evaluation metric
def accuracy(y_true, y_pred):
  # y_true = 1, y_pred = 1
  # y_true = 0, y_pred = 0
  correct_classification = torch.eq(y_true, y_pred).sum().item()
  acc = (correct_classification / len(y_pred)) * 100
  return acc

def train_step(epoch, model, data, loss_fn, optimizer):
  train_loss, train_acc = 0,0
  # model.to(device)

  for batch, (X, y) in enumerate(data):
    X,y = X.to(device), y.to(device)

    # Feedforward - it calls forward method inside Model Class
    y_pred = model(X)
    # Calculate loss
    loss = loss_fn(y_pred, y)
    train_loss += loss
    train_acc += accuracy(y, y_pred.argmax(dim=1))

    # Backpropagate
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  train_loss /= len(data)
  train_acc /= len(data)
  train_acc_history.append(train_acc)
  train_loss_history.append(train_loss)
  print(f"Epoch : {epoch} | Train Loss : {train_loss:.3f} |  Train Acc : {train_acc:.3f}")


def test_step(epoch, model, data, loss_fn, optimizer):
  test_loss, test_acc = 0,0
  # model.to(device)
  model.eval()

  with torch.inference_mode():
    for batch, (X, y) in enumerate(data):
      X,y = X.to(device), y.to(device)

      # Feedforward - it calls forward method inside Model Class
      y_pred = model(X)
      # Calculate loss
      loss = loss_fn(y_pred, y)
      test_loss += loss
      test_acc += accuracy(y, y_pred.argmax(dim=1))

    test_loss /= len(data)
    test_acc /= len(data)
    print(f"Epoch : {epoch} | Test Loss : {test_loss:.3f} |  Test Acc : {test_acc:.3f}")

In [63]:
model_resnet = models.resnet18(weights="IMAGENET1K_V1")

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 117MB/s]


In [64]:
num_features = model_resnet.fc.in_features

In [65]:
num_features

512

In [66]:
model_resnet.fc = nn.Linear(num_features, 15)
model_resnet = model_resnet.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_resnet.parameters(), lr=0.001)

In [67]:
train_acc_history = []
train_loss_history = []

epochs = 15

for epoch in tqdm(range(epochs)):
  train_step(epoch, model_resnet, training_generator, loss_fn, optimizer)
  # test_step(epoch, model_resnet, test_generator, loss_fn, optimizer)

  7%|▋         | 1/15 [00:09<02:12,  9.48s/it]

Epoch : 0 | Train Loss : 2.240 |  Train Acc : 29.665


 13%|█▎        | 2/15 [00:18<02:02,  9.41s/it]

Epoch : 1 | Train Loss : 1.831 |  Train Acc : 40.293


 20%|██        | 3/15 [00:29<01:57,  9.82s/it]

...

Epoch : 12 | Train Loss : 0.301 |  Train Acc : 89.902


 93%|█████████▎| 14/15 [02:13<00:09,  9.59s/it]

Epoch : 13 | Train Loss : 0.333 |  Train Acc : 89.300


100%|██████████| 15/15 [02:23<00:00,  9.57s/it]

Epoch : 14 | Train Loss : 0.239 |  Train Acc : 91.663
